In [1]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)
dataFrame.show(1)

providing ../data/raw/2022_place_canvas_history-000000000050.csv ...
../data/raw/2022_place_canvas_history-000000000050.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6b45c05e-d096-4f0c-9ce1-f3ed9569be63;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 77ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

+--------------------+---+----+---+-----------+
|             user_id|  x|   y|  t|pixel_color|
+--------------------+---+----+---+-----------+
|vKtmwRQwAYtw5poO4...|804|1992|  0|    #FF3881|
+--------------------+---+----+---+-----------+
only showing top 1 row



<div style="color:pink">

Idee: 
-Daten nach Nutzern aggregieren und die Zeit zwischen aufeinanderfolgenden Pixeln vergleichen[erledigt] <br>
-Zeit (Differenz) festhalten und mit dem nächsten Zeitraum vergleichen [erledigt] <br>
-Falls die Differenz abweicht, ist der Nutzer wahrscheinlich kein Bot(?) <br>

</div>

In [2]:
from src.features.feature_functions import group_dataframe_by_user
groupedFrame = group_dataframe_by_user(dataFrame)
#groupedFrame.show(5)
#brauchen wir gar nicht mehr :)

In [3]:
import pyspark.sql.functions as F
timeDiffFrame = dataFrame.alias('df1').join(dataFrame.alias('df2'),F.col('df1.user_id') == F.col('df2.user_id'),'inner')
timeDiffFrame = timeDiffFrame.where('df2.t > df1.t')
timeDiffFrame = timeDiffFrame.orderBy(['df1.user_id', 'df1.t'])
timeDiffFrame = timeDiffFrame.select(F.col('df1.user_id'),F.col('df1.t').alias('t1'),F.col('df2.t').alias('t2'),(F.col('df2.t') - F.col('df1.t')).alias('diff'))
timeDiffFrame.show(15)

+--------------------+----+----+----+
|             user_id|  t1|  t2|diff|
+--------------------+----+----+----+
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1087|2995|1908|
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1087|2995|1908|
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1087|2995|1908|
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1087|2995|1908|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/iKk3F3ZlK4sJxa...| 145|2174|2029|
|++/iKk3F3ZlK4sJxa...| 145|1201|1056|
|++/iKk3F3ZlK4sJxa...| 145| 548| 403|
+--------------------+----+----+----+
only showing top 15 rows



<div style ="color:pink">
-Differenz speichern und beim nächsten Vergleich erneut verwenden? [erledigt] <br>
-Nutzer, die das Kriterium bei einem der Vergleiche nicht erfüllen, müssten nicht erneut betrachtet werden(?) <br>
<br>
SELECT user_id, diff <br>
FROM timeDiffFrame <br>
WHERE NOT EXISTS(<br>
   SELECT *<br>
   FROM timeDiffFrame a, timeDiffFrame b<br>
   WHERE a.user_id = b.user_id<br>
   AND a.diff <> b.diff);<br>
</div>

In [4]:
timeDiffReduced = timeDiffFrame.dropDuplicates(['user_id', 't1'])
timeDiffReduced = timeDiffReduced.orderBy(['user_id', 't1'])
timeDiffReduced.show(15)

22/06/07 07:57:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:57:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:57:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:57:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:57:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:57:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:57:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:57:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:57:19 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+----+----+----+
|             user_id|  t1|  t2|diff|
+--------------------+----+----+----+
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/iKk3F3ZlK4sJxa...| 145| 548| 403|
|++/iKk3F3ZlK4sJxa...| 548| 870| 322|
|++/iKk3F3ZlK4sJxa...| 870|1201| 331|
|++/iKk3F3ZlK4sJxa...|1201|1553| 352|
|++/iKk3F3ZlK4sJxa...|1553|2174| 621|
|++/iKk3F3ZlK4sJxa...|2174|2487| 313|
|++/xB4v5Jeeacv4KO...|2037|2606| 569|
|++0X370V+dVFAmRQD...|2357|2660| 303|
|++0X370V+dVFAmRQD...|2660|2974| 314|
|++0X370V+dVFAmRQD...|2974|3291| 317|
|++0nCmngtZ68vbg6p...| 734|2690|1956|
|++0rQ+/WspcGyNaOH...|3008|3316| 308|
|++1HxWRIxcZdBaKrR...|1149|3310|2161|
+--------------------+----+----+----+
only showing top 15 rows



In [7]:
botDiff = timeDiffReduced.alias('tdr1').join(timeDiffReduced.alias('tdr2'),F.col('tdr1.user_id') == F.col('tdr2.user_id'),'inner')
botDiff = botDiff.where('tdr1.diff == tdr2.diff').where('tdr1.t2 == tdr2.t1')
botDiff = botDiff.select('tdr1.user_id', 'tdr1.diff')
botDiff = botDiff.dropDuplicates(['user_id'])
botDiff = botDiff.orderBy(['user_id'])
botDiff.show(15)

22/06/07 07:58:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:14 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+----+
|             user_id|diff|
+--------------------+----+
|++6IlQ1s33JtGR24y...| 316|
|++ZTAhuLp4CF8NxY2...| 329|
|+/8RPTGDQ4veBkUkY...| 311|
|+05i4u0UwrZ9JkLMX...| 309|
|+0AL86CS7hWfXsH5s...| 344|
|+0pWUzoKBK56oKF8g...| 305|
|+0x7bg9KgIJNGqLlu...| 303|
|+1JgW4Wly+kXochax...| 303|
|+1f+sQqThukXAPL0T...| 329|
|+2/56spvxox9vRQg8...| 385|
|+2/7IC4kXK8W1W+bU...| 337|
|+2ESl4h5Jg0ydCpZJ...| 306|
|+2J+7lTMXssxzQ5Qe...| 308|
|+2R4sYRmGPzYLNdPC...| 356|
|+2fkJNNiG9lpcsrtm...| 308|
+--------------------+----+
only showing top 15 rows



In [8]:
print(botDiff.count())

22/06/07 07:58:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 07:58:31 WARN RowBasedKeyValueBatch: Calling spill() on

14132
